### Token Added
This notebook is run under the IBM Watson Studio.  
Thus, the data insert is different from the normal data processing.

There are two steps before the major processing:  
1. Upload data assets into IBM Watson Project
2. Create token and insert into the notebook (the first cell) for the later final data accessing  



In [2]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='10308584-75a2-44b1-b5c5-d0ee0ea4840e', project_access_token='p-1ff59996d40f2a41765d5168e32aa3237a9966ec')
pc = project.project_context


In [3]:
import numpy as np
import pandas as pd
import sys

In [4]:
# System info
print(sys.version_info)

sys.version_info(major=3, minor=5, micro=5, releaselevel='final', serial=0)


### Insert data (X_train.csv, Y_train.csv)

In [5]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_bb2ff9ec7be94d32a0bcbbd5629362b2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='CWeC0la3wD3mvbpFpn2ZIzg4jQv5EZ4UTmc7ohqTOdrF',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bb2ff9ec7be94d32a0bcbbd5629362b2.get_object(Bucket='mlhelpnavigaterobots-donotdelete-pr-y9h1hr72ryyche',Key='X_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

X_train = pd.read_csv(body)


body = client_bb2ff9ec7be94d32a0bcbbd5629362b2.get_object(Bucket='mlhelpnavigaterobots-donotdelete-pr-y9h1hr72ryyche',Key='y_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Y_train = pd.read_csv(body)



In [6]:
# Display X_train data
X_train.tail(10)

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
487670,3809_118,3809,118,0.62824,-0.76917,-0.084778,0.080621,0.006636,0.109980,-0.052170,-0.63579,2.8324,-13.2950
487671,3809_119,3809,119,0.62831,-0.76912,-0.084648,0.080769,0.006267,0.165260,-0.053871,2.33030,3.7376,-13.3650
487672,3809_120,3809,120,0.62838,-0.76906,-0.084546,0.080858,0.018649,0.114640,-0.062610,0.28030,2.8864,-11.7880
487673,3809_121,3809,121,0.62845,-0.76899,-0.084540,0.080911,0.023261,0.071968,-0.096275,3.97270,3.0608,-11.2320
487674,3809_122,3809,122,0.62856,-0.76890,-0.084453,0.081021,-0.001146,0.124560,-0.101340,4.05960,3.9223,-9.3404
487675,3809_123,3809,123,0.62871,-0.76878,-0.084391,0.081093,0.003167,0.093760,-0.142740,3.27180,2.0115,-9.0063
487676,3809_124,3809,124,0.62884,-0.76868,-0.084365,0.081099,0.014994,0.032637,-0.132380,4.42750,3.0696,-8.1257
487677,3809_125,3809,125,0.62891,-0.76861,-0.084345,0.081178,-0.031184,-0.003961,-0.138940,2.70480,4.2622,-8.1443
487678,3809_126,3809,126,0.62903,-0.76850,-0.084414,0.081231,-0.069153,0.013229,-0.130210,2.54100,4.7130,-9.4435
487679,3809_127,3809,127,0.62915,-0.76839,-0.084441,0.081284,-0.042769,0.034049,-0.125800,0.82391,4.2751,-10.4980


In [7]:
# Display Y_train data
Y_train.tail(10)

,series_id,group_id,surface
3800,3800,68,fine_concrete
3801,3801,59,hard_tiles_large_space
3802,3802,55,tiled
3803,3803,45,wood
3804,3804,66,tiled
3805,3805,55,tiled
3806,3806,67,wood
3807,3807,48,fine_concrete
3808,3808,54,tiled
3809,3809,56,soft_pvc


In [8]:
X_train.shape

(487680, 13)

In [9]:
Y_train.shape

(3810, 3)

In [10]:
#list(Y_train.columns.values)
Y_train.columns.tolist()

['series_id', 'group_id', 'surface']

Merge two dataframe (X_train and Y_train) into one with the foreign key `series_id`.

In this competition, one series has the same floor surface.

In [11]:
D_train = pd.merge(X_train, Y_train, on='series_id')
D_train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,group_id,surface
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532,13,fine_concrete
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128,13,fine_concrete
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267,13,fine_concrete
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960,13,fine_concrete
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410,13,fine_concrete


In [12]:
D_train.shape

(487680, 15)

In [13]:
D_train.surface.value_counts()

concrete                  99712
soft_pvc                  93696
wood                      77696
tiled                     65792
fine_concrete             46464
hard_tiles_large_space    39424
soft_tiles                38016
carpet                    24192
hard_tiles                 2688
Name: surface, dtype: int64

In [14]:
D_train.surface.describe()

count       487680
unique           9
top       concrete
freq         99712
Name: surface, dtype: object

Want the unique values in the `surface` column:

In [15]:
keys = D_train.surface.unique().tolist()
keys

['fine_concrete',
 'concrete',
 'soft_tiles',
 'tiled',
 'soft_pvc',
 'hard_tiles_large_space',
 'carpet',
 'hard_tiles',
 'wood']

In [22]:
values = range(len(keys))
list(values)

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [26]:
value_dic = dict(zip(keys, values))
value_dic

{'carpet': 6,
 'concrete': 1,
 'fine_concrete': 0,
 'hard_tiles': 7,
 'hard_tiles_large_space': 5,
 'soft_pvc': 4,
 'soft_tiles': 2,
 'tiled': 3,
 'wood': 8}

In [28]:
value_dic['carpet']

6

In [30]:
D_train.surface = [value_dic[i] for i in D_train.surface]

In [35]:
D_train.tail()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,group_id,surface
487675,3809_123,3809,123,0.62871,-0.76878,-0.084391,0.081093,0.003167,0.093760,-0.14274,3.27180,2.0115,-9.0063,56,4
487676,3809_124,3809,124,0.62884,-0.76868,-0.084365,0.081099,0.014994,0.032637,-0.13238,4.42750,3.0696,-8.1257,56,4
487677,3809_125,3809,125,0.62891,-0.76861,-0.084345,0.081178,-0.031184,-0.003961,-0.13894,2.70480,4.2622,-8.1443,56,4
487678,3809_126,3809,126,0.62903,-0.76850,-0.084414,0.081231,-0.069153,0.013229,-0.13021,2.54100,4.7130,-9.4435,56,4
487679,3809_127,3809,127,0.62915,-0.76839,-0.084441,0.081284,-0.042769,0.034049,-0.12580,0.82391,4.2751,-10.4980,56,4


Testing data is splited from the whole training data.  
Because there is no testing data to test the model accuracy.  
And, the provided testing data has no result.

In [41]:
from sklearn.cross_validation import train_test_split

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [42]:
train, test = train_test_split(D_train, test_size=0.2, random_state=0, stratify=D_train.surface)

In [44]:
train.shape

(390144, 15)

In [45]:
test.shape

(97536, 15)

In [48]:
train.shape[0] + test.shape[0]

487680

### Export testing and training data (train, test)

In [49]:
project.save_data(data=train.to_csv(index=False), file_name='train.csv', overwrite=True)

{'asset_id': '49c40327-7654-4490-8071-db88b94ddcf7',
 'bucket_name': 'mlhelpnavigaterobots-donotdelete-pr-y9h1hr72ryyche',
 'file_name': 'train.csv',
 'message': 'File saved to project storage.'}

In [50]:
project.save_data(data=test.to_csv(index=False), file_name='test.csv', overwrite=True)

{'asset_id': '2372092e-d66e-4813-a8f9-361065811ba3',
 'bucket_name': 'mlhelpnavigaterobots-donotdelete-pr-y9h1hr72ryyche',
 'file_name': 'test.csv',
 'message': 'File saved to project storage.'}